<a href="https://colab.research.google.com/github/mdonbruce/AspNetDocs/blob/master/Module02_NumPy_Vectorized_Order_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 2 — Python & NumPy Foundations
## Hands-On In-Class Activity: Vectorized Order Analytics with NumPy

**Scenario:** You are a junior data analyst helping an e-commerce team understand revenue, returns, and shipping performance.
Your job is to use **NumPy** to compute key metrics efficiently (vectorization) and to spot how **data quality issues** can impact results.

**Files included:**
- `module02_numpy_orders.csv` (≈25,000 rows)
- `starter_script_module02_numpy.py`

**What you’ll practice (Module 2 alignment):**
- Python variables, data types, and control structures (briefly)
- Creating and manipulating NumPy arrays
- Indexing, slicing, boolean masking
- Vectorized math + aggregations (`sum`, `mean`, `min`, `max`)

**Deliverable (group):** A short write-up (1–2 pages) with code snippets + screenshots of outputs.


## Step 1 — Setup & Load Data
Run the next cell to import libraries and load the dataset.

In [1]:
import numpy as np
import pandas as pd

DATA_PATH = "module02_numpy_orders.csv"
df = pd.read_csv(DATA_PATH)
df.head()

,order_id,order_date,customer_segment,region,channel,product_category,unit_price,quantity,discount_rate,shipping_days,returned,satisfaction_score
0,100000,2025-02-14,Corporate,West,Partner,Software,137.89,3,0.166,3.7,0,4.6
1,100001,2025-12-06,Consumer,Midwest,Web,Software,60.15,3,0.037,2.2,0,4.9
2,100002,2025-10-10,Home Office,South,Store,Accessories,42.55,3,0.069,2.2,0,4.3
3,100003,2025-05-04,Corporate,Northeast,Store,Office Supplies,18.06,4,0.088,1.0,0,5.0
4,100004,2025-01-17,Consumer,South,Store,Office Supplies,41.24,12,0.076,1.5,0,4.6


## Step 2 — Quick Dataset Inspection
Answer as a group:
1. How many rows and columns are there?
2. Which columns are numeric vs categorical?
3. Do you see missing values?

In [2]:
df.shape, df.dtypes

((25000, 12),
 order_id                int64
 order_date             object
 customer_segment       object
 region                 object
 channel                object
 product_category       object
 unit_price            float64
 quantity                int64
 discount_rate         float64
 shipping_days         float64
 returned                int64
 satisfaction_score    float64
 dtype: object)

In [3]:
df.isna().mean().sort_values(ascending=False).head(10)

,0
discount_rate,0.005
unit_price,0.005
order_id,0.000
order_date,0.000
region,0.000
customer_segment,0.000
product_category,0.000
channel,0.000
quantity,0.000
shipping_days,0.000


## Step 3 — Basic Cleaning (Minimal)
The dataset includes a small number of missing values and outliers.

**In real projects:** cleaning decisions depend on context. For this activity, we’ll do **simple** cleaning so NumPy computations run smoothly.

**Group decision:** If you change these strategies, document what you did and why.

In [4]:
num_cols = ["unit_price", "discount_rate", "shipping_days", "satisfaction_score"]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Simple imputations
df["unit_price"] = df["unit_price"].fillna(df["unit_price"].median())
df["discount_rate"] = df["discount_rate"].fillna(0.0)

df[num_cols].describe().T

,count,mean,std,min,25%,50%,75%,max
unit_price,25000.0,118.110539,172.160013,4.72,23.790,89.730,178.290,9237.060
discount_rate,25000.0,0.070597,0.026316,0.00,0.053,0.068,0.085,0.208
shipping_days,25000.0,2.587104,0.891167,0.50,2.000,2.600,3.100,14.800
satisfaction_score,25000.0,4.230052,0.526139,1.80,3.900,4.300,4.600,5.000


## Step 4 — Convert Key Columns to NumPy Arrays
This is the **core** of the activity: compute metrics using **NumPy**, not pandas groupby.

**Tip:** Prefer `.to_numpy()` and vectorized operations over Python loops.

In [5]:
order_id = df["order_id"].to_numpy(dtype=int)
unit_price = df["unit_price"].to_numpy(dtype=float)
quantity = df["quantity"].to_numpy(dtype=int)
discount_rate = df["discount_rate"].to_numpy(dtype=float)
returned = df["returned"].to_numpy(dtype=int)
shipping_days = df["shipping_days"].to_numpy(dtype=float)
satisfaction = df["satisfaction_score"].to_numpy(dtype=float)

category = df["product_category"].to_numpy(dtype=str)
channel = df["channel"].to_numpy(dtype=str)
region = df["region"].to_numpy(dtype=str)

order_id[:5], unit_price[:5], quantity[:5], discount_rate[:5]

(array([100000, 100001, 100002, 100003, 100004]),
 array([137.89,  60.15,  42.55,  18.06,  41.24]),
 array([ 3,  3,  3,  4, 12]),
 array([0.166, 0.037, 0.069, 0.088, 0.076]))

## Step 5 — Vectorized Revenue Computation
Define:
- **gross** = unit_price × quantity
- **net** = gross × (1 − discount_rate)

Then answer:
1. What is the **total** net revenue?
2. What is the **average** net revenue per order?
3. What are the **top 5** orders by net revenue?

In [6]:
gross = unit_price * quantity
net = gross * (1.0 - discount_rate)

net_total = net.sum()
net_avg = net.mean()

# Top 5 orders by net revenue
top_idx = np.argsort(net)[-5:][::-1]
top5 = list(zip(order_id[top_idx].tolist(), np.round(net[top_idx], 2).tolist()))

net_total, net_avg, top5

(np.float64(7088416.108599999),
 np.float64(283.53664434399997),
 [(123724, 23829.77),
  (107930, 16790.35),
  (116549, 15583.82),
  (116136, 15509.63),
  (109914, 14564.59)])

## Step 6 — Boolean Masking: Returns & Their Impact
Compute:
- Overall **return rate**
- Average net revenue for returned vs not returned
- Does **shipping_days** appear related to returns?

**Note:** This is not a causal claim—just pattern detection.

In [7]:
return_rate = returned.mean()

mask_ret = returned == 1
net_ret_avg = net[mask_ret].mean()
net_not_ret_avg = net[~mask_ret].mean()

ship_ret_avg = shipping_days[mask_ret].mean()
ship_not_ret_avg = shipping_days[~mask_ret].mean()

return_rate, net_ret_avg, net_not_ret_avg, ship_ret_avg, ship_not_ret_avg

(np.float64(0.0444),
 np.float64(326.93251522522525),
 np.float64(281.5203439388865),
 np.float64(2.598828828828829),
 np.float64(2.586559229803265))

## Step 7 — Group Aggregation Without `groupby`
### Task A: Net revenue by category
Use `np.unique` and boolean masks.

### Task B: Average shipping days by channel
Same idea: compute a mean per group.


In [8]:
cats = np.unique(category)
net_by_cat = {}
for c in cats:
    m = (category == c)
    net_by_cat[c] = float(np.round(net[m].sum(), 2))

chans = np.unique(channel)
ship_by_chan = {}
for ch in chans:
    m = (channel == ch)
    ship_by_chan[ch] = float(np.round(shipping_days[m].mean(), 2))

net_by_cat, ship_by_chan

({np.str_('Accessories'): 453979.53,
  np.str_('Electronics'): 3922918.06,
  np.str_('Furniture'): 1403308.9,
  np.str_('Office Supplies'): 837696.01,
  np.str_('Software'): 470513.6},
 {np.str_('Mobile'): 2.46,
  np.str_('Partner'): 3.79,
  np.str_('Store'): 1.77,
  np.str_('Web'): 2.88})

## Step 8 — Indexing & Slicing Mini-Challenge
1. Slice the first **1,000** orders and compute their total net revenue.
2. Randomly sample **500** orders and compute the return rate in the sample.
3. Create a mask for **high-value orders** (net > 500) and report:
   - how many there are
   - their average satisfaction score


In [9]:
first_1000_total = net[:1000].sum()

rng = np.random.default_rng(0)
sample_idx = rng.choice(len(net), size=500, replace=False)
sample_return_rate = returned[sample_idx].mean()

high_value = net > 500
high_count = int(high_value.sum())
high_satisfaction_avg = float(np.round(satisfaction[high_value].mean(), 2))

first_1000_total, sample_return_rate, high_count, high_satisfaction_avg

(np.float64(294755.17906999995), np.float64(0.04), 3893, 4.23)

## Step 9 — Reflection Prompts (Group)
Use your results to answer:
1. What key insights were revealed through your evaluation process?
2. Which metric(s) were most useful for understanding performance, and why?
3. What were the main challenges faced during the analysis (e.g., missing values, outliers, logic errors)?
4. How might data issues (missing values/outliers) change a business decision?
5. If you had more time, what steps would improve reliability (validation checks, better imputations, monitoring)?


## Optional Extension (If Time)
- Create a **simple rule-based** flag for potentially risky orders (e.g., net > 2000 AND shipping_days > 6).
- Compare satisfaction between flagged and non-flagged orders.
- Discuss whether your rule is fair, and what might bias it.
